In [13]:
import pandas as pd

In [14]:
tuning_result = pd.read_csv('../models/tuning/rfr_1.csv')

In [15]:
tuning_result.sort_values(['Val score'], ascending = False)

,n_estimators,max_depth,random_state,ccp_alpha,max_samples,max_features,Train score,Val score,Test score,Time
173,100.0,25.0,19421220.0,0.0001,0.50,0.75,0.6031,0.4889,0.4541,70.38
53,50.0,25.0,19421220.0,0.0001,0.50,0.75,0.6003,0.4876,0.4501,35.04
149,100.0,10.0,19421220.0,0.0001,0.50,0.75,0.5881,0.4873,0.4553,36.62
197,100.0,50.0,19421220.0,0.0001,0.50,0.75,0.6044,0.4872,0.4580,129.56
221,100.0,100.0,19421220.0,0.0001,0.50,0.75,0.6044,0.4872,0.4580,70.40
...,...,...,...,...,...,...,...,...,...,...
140,100.0,5.0,19421220.0,10000.0000,0.50,0.50,-0.0000,-0.0000,-0.0000,12.81
141,100.0,5.0,19421220.0,10000.0000,0.50,0.75,-0.0000,-0.0000,-0.0000,19.09
142,100.0,5.0,19421220.0,10000.0000,0.75,0.50,-0.0000,-0.0000,-0.0000,17.00
143,100.0,5.0,19421220.0,10000.0000,0.75,0.75,-0.0000,-0.0000,-0.0000,25.26
